# Graph partitioning

An implementation of graph partitioning using an Ising model.  

## Problem description
Consider a graph $G=(V,E)$ with an even Number $n=|V|$ of vertices and edges $E$. What is an partitioning of $G$ into *two subsets*, such that both subsets have the *same number of vertices* $n\over 2$ and the *number of connecting edges between the sets is minimized*.

## Ising model
Associate with each vertice a value $s_i=\pm 1\;(i=1,\dots,n)$ where the two different values denote in which of the subsets each vertice lies.  
The classical hamiltonian $H=AH_A + BH_B$ consists of two parts, where

$$
H_A = (\sum_i s_i)^2 = n + 2\sum_{i,j;i<j}s_is_j
$$

is the constraint for the sets having the same size and should be $0$ for ground states and the square taking part of making more down spins also a penalty and 

$$
H_B = \sum_{ij\in E} (1 - s_is_j)
$$

is the function to be minimized. If an edge connects two vertices of the same group the term evaluates to $0$, if it connects two vertices of different groups the term becomes $2$ and suc edges therefore get penalized.  
(Note that for choosing $B<0$ we get the corresponding problem of maximizing the connecting edges.)

For finding a lower bound for $A$ consider the maximal gain in energy for breaking the constraint by flipping one spin in an even partition $\Delta H_B = 2B\min({d,n/2})$ where $d$ is the maximal number of vertices one vertice in the graph can be connected to. The minimal penalty in energy for such violations is $\Delta H_A = 4A$. We need $ \Delta H_B \leq \Delta H_A $ in order for the penalty to compensate any energy gain by breaking the constraint.

So as an lower bound for $A$ we get

$$\begin{equation}
A \geq \frac{B}{2}\min({d,n/2})
\end{equation}$$

## Implementation

In [2]:
# import packages
import dimod

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave.preprocessing import roof_duality
import neal

import numpy as np
import igraph as ig

import os

In [3]:
# setting the sampler for later
sampler = neal.SimulatedAnnealingSampler()

In [9]:
#################
# Define functions
#################

def generate_graph(verticenumber, edgenumber):
    """generates a random graph of given size"""
    g = ig.Graph.Erdos_Renyi(n=verticenumber, m=edgenumber)
    g.vs["index"] = list(range(1, verticenumber+1))
    g.vs["spin"] = [-1 for i in list(range(1, verticenumber+1))]
    # wether the edge connects vertices of the same group
    g.es["connection"] = [1 for i in list(range(1, verticenumber+1))]
    return(g)


def update_graph(g, sample):
    """"updates the attributes of the graph according to a sample of spins"""
    # upate spins
    for i in range(len(sample)):
        g.vs[i]["spin"] = sample[i]
    # update edges
    connectionlist = []
    for edge in g.es():
        source_spin = g.vs["spin"][edge.source]
        target_spin = g.vs["spin"][edge.target]
        # if source and target have the same spin the edge is not "connecting"
        connection = target_spin*source_spin
        connectionlist.append(connection)
    g.es["connection"] = connectionlist
    return(g)   # not neccessary, the object gets altered. 
                # but I remain this in order not to have to 
                # change everything where I use g = update_graph(g, sample)

def init_graph(g):
    """initiates the attributes of a graph to initial values"""
    g.vs["index"] = list(range(1, len(g.vs())+1))
    g.vs["spin"] = [-1 for i in list(range(1, len(g.vs())+1))]
    g.es["connection"] = [1 for i in list(range(1, len(g.vs())+1))]
    return(g) # again in theory not neccessary anymore


def plot_graph(g, save_as=None):
    """plots a graph with visualization of the spins and connecting edges"""
    visual_style = {}
    visual_style["vertex_label"] = g.vs["index"]
    visual_style["vertex_color"] = [
        "green" if spin == +1 else "red" for spin in g.vs["spin"]]
    visual_style["title"] = "test"
    visual_style["edge_width"] = [4 if connection ==
                                  1 else 2 for connection in g.es["connection"]]
    visual_style["edge_color"] = ["black" if connection ==
                                  1 else "red" for connection in g.es["connection"]]
    plot = ig.plot(g, save_as, bbox=(0, 0, 300, 400), **visual_style)   
    # the iGraph plot gets a dictonary of keyword options
    return(plot)


def graph_partitioning_make_bqm(g, AB_difference=10):
    """given a graph calculates the coefficients of the corresponding bqm"""
    edgelist = g.get_edgelist()  # edges as tuples in a set
    # vertices are just the numbers from 1 up 
    # to n = indices of the spin variables = keys of the term
    n = len(g.vs())
    deg = max(g.degree())
    B = 50
    A = np.ceil(B/4 * min(2*deg, n)) + AB_difference    
    #for experimenting one can change here the difference between A and B

    linear = {}     # no linear terms

    quadratic = {}

    constant = A*n + B*len(g.es())

    # Calculate the biases and couplers according to above equations
    # contribution of H_A
    for j in range(n):  # j ranging from 0 to n-1 
        for i in range(j):  # i ranging from 0 to j-1 -> i<j
            quadratic[i, j] = 2*A
    # contribution of H_B
    for (i, j) in edgelist:
        quadratic[i, j] += -B

    bqm = dimod.BinaryQuadraticModel(
        linear, quadratic, offset=constant, vartype=dimod.SPIN)
    return(bqm)


def graph_partitioning(g=None, verticenumber=6, edgenumber=7,
                       AB_difference=10, iterates=100, show_numbers=False,
                       name="new_example"
                       ):
    """" This is the main function to be used in the end.
    parameters:
        g: is the graph supposed to be partitioned. If left empty a random one will be generated
        verticenumber, edgenumber: number of vertices, edges used when generating a random graph
        AB_difference: difference of coefficients A and B for experimenting
        iterates: number of iterations to be run on the QA
        show_numbers: wether to print the result
        name: name of the example and the folder weher to put the results, figures etc.
    output:
        saves the figures and data from DWave in a folder given by name
    """
    # generate the folder weher to save the results if it does not exist
    folder = f"graph_examples\\{name}"
    if not(os.path.isdir(folder)):
        os.makedirs(folder)

    # generate a random graph or initate the attributes of a given one
    if g == None:
        g = generate_graph(verticenumber, edgenumber)
    else:
        g = init_graph(g)

    # generate the bqm and sample it on the DWave QA
    bqm = graph_partitioning_make_bqm(g, AB_difference=AB_difference)
    result = sampler.sample(bqm, num_reads=iterates)

    # save the results
    with open(f"{folder}\\result.txt", "w") as outfile:
        outfile.write(f"results from DWave:\n{result}")
    with open(f"{folder}\\info.txt", "w") as outfile:
        outfile.write(str(result.info))
        # for t in result.info["timing"]:
        #     time = result.info["timing"][t]
        #     outfile.write(f"{t}: {time}\n")
    with open(f"{folder}\\adjecency_matrix.txt", "w") as outfile:
        outfile.write(str(g.get_adjacency()))
    
    # print the results
    if show_numbers:
        print(f"results form DWave:\n {result}")

    # collect the samples with the lowest energy
    best_energy = result.record["energy"][0]
    best_samples = []
    for row in result.record:
        if row["energy"] == best_energy:
            best_samples.append(row["sample"])
            continue
        break
    
    # check if a solution satisfying the condition 
    # (same number of vertices in both groups) was found
    if sum(best_samples[0]) != 0:
        print("no solution satisfying the condition found")
        print("unbalance =", sum(best_samples[0]))

    # from the collected samples, 
    # generate the graphs with the according spins
    possible_graphs = []
    for i in range(len(best_samples)):
        new_copy = g.copy()
        possible_graphs.append(update_graph(new_copy, best_samples[i]))

    # save the plots of the graphs
    pathnum = 1
    for g in possible_graphs:
        path = f"{folder}\\part_possability{pathnum}.png"
        plot_graph(g, save_as=path)
        pathnum += 1

    # return(result)


In [5]:
# get famous graphs with ig.Graph.Famous("name")
# ig.Graph.Tree(size, children)

In [10]:
g = ig.Graph.Famous('Cubical')
graph_partitioning(g=g)

- The Herschel graph is the smallest nonhamiltonian polyhedral graph. It is the unique such graph on 11 nodes, and has 18 edges.
- Smallestcyclicgroup - A smallest nontrivial graph whose automorphism group is cyclic. It has 9 vertices and 15 edges.
- tree(16,2)
- Cubical - The Platonic graph of the cube. A convex regular polyhedron with 8 vertices and 12 edges.